In [1]:
import graphlab

In [2]:
products = graphlab.SFrame("amazon_baby.csv/amazon_baby.csv")

This non-commercial license of GraphLab Create for academic use is assigned to edward30@163.com and will expire on September 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Edward\AppData\Local\Temp\graphlab_server_1503062296.log.0


Finished parsing file C:\Users\Edward\Anaconda2\Scripts\Demo\Classification\amazon_baby.csv\amazon_baby.csv

Parsing completed. Parsed 100 lines in 0.757043 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105953 lines. Lines per second: 95276

Finished parsing file C:\Users\Edward\Anaconda2\Scripts\Demo\Classification\amazon_baby.csv\amazon_baby.csv

Parsing completed. Parsed 183531 lines in 1.47208 secs.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4


# Fill n/a value

In [5]:
products['review'] = products['review'].apply(lambda str : 'NaN' if str == '' else str)

In [6]:
products[products['review']=='']

name,review,rating


# Perform text cleaning

In [7]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

In [7]:
len(products)

183531

# Extract sentiment

In [8]:
products = products[products['rating'] != 3]

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [10]:
products

name,review,rating,review_clean,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,it came early and was notdisappointed i love ...,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,Very soft and comfortableand warmer than it ...,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,This is a product wellworth the purchase I ...,1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,All of my kids have criednonstop when I tried to ...,1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,When the Binky Fairy cameto our house we didnt ...,1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,Lovely book its boundtightly so you may no ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,Perfect for new parentsWe were able to keep ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,A friend of mine pinnedthis product on Pinte ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,I love this journal andour nanny uses it ...,1


# Split into training and test sets

In [12]:
train_data, test_data = products.random_split(.8, seed=1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [15]:
import sklearn

In [16]:
sentiment_label = sklearn.linear_model.LogisticRegression()

In [17]:
len(train_data['sentiment'])

133416

In [18]:
len(train_data['review_clean'])

133416

In [19]:
sentiment_label.fit(train_matrix,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
coef = sentiment_label.coef_

In [22]:
coef

array([[ -1.23944706e+00,   1.46839506e-04,   2.60987406e-02, ...,
          1.20964486e-02,   2.99346998e-03,  -6.03713863e-05]])

In [23]:
sum = 0
i=0
while i < coef[0].size:
    if(coef[0][i]>=0):
        sum+=1
    i+=1

In [24]:
print sum

86000


# Make prediction with logistic regression

In [25]:
sample_test_data = test_data[10:13]
print sample_test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |   5    |
| Wall Decor Removable Decal... | Would not purchase again o... |   2    |
| New Style Trailing Cherry ... | Was so excited to get this... |   1    |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|          review_clean         | sentiment |
+-------------------------------+-----------+
| Absolutely love it and all... |     1     |
| Would not purchase again o... |     -1    |
| Was so excited to get this... |     -1    |
+-------------------------------+-----------+
[3 rows x 5 columns]



In [26]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_label.decision_function(sample_test_matrix)
print scores

[  5.58246223  -3.16015917 -10.4308965 ]


In [27]:
scores = sentiment_label.predict(sample_test_matrix)
print scores

[ 1 -1 -1]


# Find the most positive review

In [28]:
test_data['scores'] = sentiment_label.decision_function(test_matrix)

In [29]:
test_data

name,review,rating,review_clean,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,I love this journal andour nanny uses it ...,1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5,I love this littlecalender you can keep ...,1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5,I had a hard time findinga second year calendar ...,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4,One of babys first andfavorite books and it is ...,1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5,My son loved this book asan infant It was per ...,1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5,Our baby loves this bookhas loved it for a while ...,1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2,This bear is absolutelyadorable and I would ...,-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5,I bought two for recentbaby showers The boo ...,1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5,I searched high and lowfor a first year cale ...,1


In [73]:
print_rows(num_rows=20, num_columns=10)

NameError: name 'print_rows' is not defined

In [30]:
(test_data.sort('scores', ascending=False))['name','scores'][10:20]

name,scores
Buttons Cloth DiaperCover - One Size - 8 ...,40.4490022995
Graco FastAction FoldJogger Click Connect ...,39.8136930308
Freemie Hands-FreeConcealable Breast Pump ...,39.0213161858
"Baby Jogger City Mini GTSingle Stroller, ...",38.3277129336
"Evenflo 6 Pack ClassicGlass Bottle, 4-Ounce ...",38.0637640157
"Fisher-Price Cradle 'NSwing, My Little ...",37.7603822305
"Britax DecathlonConvertible Car Seat, ...",36.5987964871
Ikea 36 Pcs Kalas KidsPlastic BPA Free ...,36.5352177184
Summer Infant Wide ViewDigital Color Video ...,35.8246870664
"Baby Jogger City Mini GTDouble Stroller, ...",35.6080914826


In [31]:
(test_data.sort('scores', ascending=True))['name','scores'][10:20]

name,scores
Ellaroo Mei Tai BabyCarrier - Hershey ...,-21.5088632048
Cosco Alpha Omega EliteConvertible Car Seat ...,-21.3914584601
"Peg-Perego Tatamia HighChair, White Latte ...",-21.0886636989
Belkin WeMo Wi-Fi BabyMonitor for Apple iPh ...,-21.0641976322
Chicco Cortina KeyFit 30Travel System in ...,-21.0485837828
NUK Cook-n-Blend BabyFood Maker ...,-20.9732407768
VTech Communications Safe&amp; Sound Digital A ...,-20.896225209
Safety 1st Deluxe 4-in-1Bath Station ...,-20.606928155
"Thirsties Hemp Inserts 2Pack, Small 6-18 Lbs ...",-20.3872163223
"Regalo My Cot PortableBed, Royal Blue ...",-20.1581053045


In [32]:
print scores.sort(axis=0)

None


In [33]:
print scores[-10:-1]

[-1 -1]


In [34]:
test_data['result'] = sentiment_label.predict(test_matrix)

In [35]:
test_data

name,review,rating,review_clean,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,I love this journal andour nanny uses it ...,1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5,I love this littlecalender you can keep ...,1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5,I had a hard time findinga second year calendar ...,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4,One of babys first andfavorite books and it is ...,1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5,My son loved this book asan infant It was per ...,1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5,Our baby loves this bookhas loved it for a while ...,1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2,This bear is absolutelyadorable and I would ...,-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5,I bought two for recentbaby showers The boo ...,1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5,I searched high and lowfor a first year cale ...,1


In [36]:
len(test_data)

33336

In [37]:
len(test_data[test_data['sentiment']==test_data['result']])

31082

In [38]:
r =31082/33336.0

In [39]:
print r

0.932385409167


# Learn another classifier with fewer words

In [87]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [88]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [89]:
simple_model = sklearn.linear_model.LogisticRegression()

In [90]:
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
simple_model_coef_table = graphlab.SFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [102]:
simple_model_coef_table.sort('coefficient',ascending=False)[10:20]

coefficient,word
-0.209562864534,less
-0.320556236735,product
-0.362166742274,would
-0.511379631799,even
-0.621168773641,work
-0.898030737715,money
-1.65157634496,broke
-2.03369861394,waste
-2.10933109032,return
-2.3482982195,disappointed


In [98]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

In [99]:
simple_model.intercept_

array([ 1.29937369])

In [104]:
train_data['result1'] = simple_model.predict(train_matrix_word_subset)

In [108]:
train_data['result'] = sentiment_label.predict(train_matrix)

In [107]:
len(train_data[train_data['sentiment']==train_data['result1']])

115648

In [109]:
len(train_data[train_data['sentiment']==train_data['result']])

129162

In [110]:
test_data['result1'] = simple_model.predict(test_matrix_word_subset)

In [111]:
len(test_data[test_data['sentiment']==test_data['result1']])

28981

In [112]:
print 28981/33336.0

0.869360451164
